# Capstone Project - The Battle of Neighborhoods in  Paris (Week 2)

## Opening a new concept of sushi restaurent francises in Paris

### 1. Import libraries

In [227]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# to get coordinates import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library import folium 

print("Libraries imported.")

Libraries imported.


### 2- Parsing the web html file with BeautifulSoup package

In [228]:
#my_url='https://www.spacelist.ca/listings/on/toronto/retail/for-lease'
my_url='https://www.annuaire-administration.com/code-postal/departement/paris.html#liste-communes-departement-Paris'

In [229]:
page=requests.get(my_url)


In [230]:
# parse data from the html into a beautifulsoup object

url = 'https://www.annuaire-administration.com/code-postal/departement/paris.html#liste-communes-departement-Paris' 
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
# Parsing links from html file
for link in soup.find_all('a'):
    print(link.get('href'))

https://www.annuaire-administration.com/
tel:118007
https://www.annuaire-administration.com/
https://www.annuaire-administration.com/mairie/
https://www.annuaire-administration.com/commune/
https://www.annuaire-administration.com/code-postal/
https://www.annuaire-administration.com/code-postal/region/
https://www.annuaire-administration.com/code-postal/departement/
https://www.annuaire-administration.com/
https://www.annuaire-administration.com/code-postal/
https://www.annuaire-administration.com/code-postal/departement/
https://www.annuaire-administration.com/code-postal/departement/paris.html
#codes-postaux-departement-Paris
https://www.annuaire-administration.com/code-postal/departement/#code-postal-carte-departements
https://www.annuaire-administration.com/code-postal/departement/#code-postal-liste-departement
tel:118007
#departement-Paris
#liste-codes-postaux-departement-Paris
https://www.annuaire-administration.com/commune/paris-75000.html
https://www.annuaire-administration.com/

In [231]:

 
#soup.find_all(class_="hotelcarac")

results=soup.find_all('tr')
nrows=len(results)
nrows
results[9].text

'\n\nCode Postal 75001\n\n\nParis 1er Arrondissement\n\n'

In [232]:

#results[29:45]
header=results[9].text.split('\n')
header


['', '', 'Code Postal 75001', '', '', 'Paris 1er Arrondissement', '', '']

In [233]:

Postcode=results[9].text.split('\n')[2]
Postcode

'Code Postal 75001'

In [234]:
neighbourhood=results[9].text.split('\n')[5]
neighbourhood

'Paris 1er Arrondissement'

In [235]:

records2 =[]
n=9
while n < 29 :
    #Postcode=results[n].text.split('\n')[2]
    neighbourhood=results[n].text.split('\n')[5]
    records2.append((neighbourhood))
    n=n+1

df2=pd.DataFrame(records2, columns=['neighbourhood'])
records =[]
n=9
while n < 29 :
    
    #neighbourhood=results[n].text.split('\n')[5]
    
    Postcode=results[n].text.split('\n')[2].strip('Code Postal')
    
#Borough=results[n].text.split('\n')[4]
    records.append((Postcode))
    n=n+1

df1=pd.DataFrame(records, columns=['PostalCode'])

d = {'PostalCode': df1['PostalCode'],'Neighbourhood': df2['neighbourhood']
    }
df = pd.DataFrame(d)
df

,PostalCode,Neighbourhood
0,75001,Paris 1er Arrondissement
1,75002,Paris 2e Arrondissement
2,75003,Paris 3e Arrondissement
3,75004,Paris 4e Arrondissement
4,75005,Paris 5e Arrondissement
5,75006,Paris 6e Arrondissement
6,75007,Paris 7e Arrondissement
7,75008,Paris 8e Arrondissement
8,75009,Paris 9e Arrondissement
9,75010,Paris 10e Arrondissement


In [236]:

df.shape



(20, 2)

In [237]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

# All requested packages already installed.



### 3. Get the geographical coordinates

In [238]:

!pip install geocoder
import geocoder

In [239]:

address =df['PostalCode'][19]
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
    


In [240]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris,France'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [241]:
# call the function to get the coordinates, store in a new list using list comprehension
geocoords = [ get_latlng(df) for df in df["PostalCode"].tolist() ]
geocoords

[[48.86341500000003, 2.3367627770000468],
 [48.86771500000003, 2.3430372260000354],
 [48.86256000000003, 2.3590470010000217],
 [48.85427500000003, 2.3614671030000522],
 [48.84533000000005, 2.3519010630000707],
 [48.84917500000006, 2.331986941000025],
 [48.85462000000007, 2.3130611620000536],
 [48.87293500000004, 2.3112484850000214],
 [48.87699500000008, 2.337893106000024],
 [48.87615487100004, 2.3623300000000427],
 [48.85977500000007, 2.378126286000054],
 [48.83510000000007, 2.4449257000000557],
 [48.83009000000004, 2.362282164000021],
 [48.82979500000005, 2.3238260650000484],
 [48.84173268100005, 2.296976166000036],
 [48.857120000000066, 2.255970651000041],
 [48.88707000000005, 2.3062929050000207],
 [48.89186500000005, 2.3480935560000376],
 [48.88394500000004, 2.385625201000039],
 [48.86239000000006, 2.400828456000056]]

In [242]:
df_geocoords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [243]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_geocoords['Latitude']
df['Longitude'] = df_geocoords['Longitude']
df

,PostalCode,Neighbourhood,Latitude,Longitude
0,75001,Paris 1er Arrondissement,48.863415,2.336763
1,75002,Paris 2e Arrondissement,48.867715,2.343037
2,75003,Paris 3e Arrondissement,48.862560,2.359047
3,75004,Paris 4e Arrondissement,48.854275,2.361467
4,75005,Paris 5e Arrondissement,48.845330,2.351901
5,75006,Paris 6e Arrondissement,48.849175,2.331987
6,75007,Paris 7e Arrondissement,48.854620,2.313061
7,75008,Paris 8e Arrondissement,48.872935,2.311248
8,75009,Paris 9e Arrondissement,48.876995,2.337893
9,75010,Paris 10e Arrondissement,48.876155,2.362330


In [244]:
df.shape

(20, 4)

In [245]:
!pip install folium
import folium

In [246]:
address = 'Paris, France'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris, France {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris, France 48.8566101, 2.3514992.


### 4. Create a map of Paris with neighborhoods  

In [247]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_paris)  
    
map_paris

### 5. Use the Foursquare API to explore the neighborhoods 

In [248]:
CLIENT_ID = 'LR2P1ZVI53BYPFRT13RMH1O2DAH33KAYKNEX5FWILOY0SXCT' 
CLIENT_SECRET = 'SLOTTAXIDVN2LFEKIN3XOBVBTYX05SL40E0SWGVPGGM1NRNQ' 
VERSION = '20180605'

In [249]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [250]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1978, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Paris 1er Arrondissement,48.863415,2.336763,Jardin du Palais Royal,48.864941,2.337728,Garden
1,Paris 1er Arrondissement,48.863415,2.336763,Palais Royal,48.863236,2.337127,Historic Site
2,Paris 1er Arrondissement,48.863415,2.336763,Comédie-Française,48.863088,2.336612,Theater
3,Paris 1er Arrondissement,48.863415,2.336763,Place du Palais Royal,48.862523,2.336688,Plaza
4,Paris 1er Arrondissement,48.863415,2.336763,Ellsworth,48.865528,2.337057,French Restaurant


In [251]:

venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
Paris 10e Arrondissement,100,100,100,100,100,100
Paris 11e Arrondissement,100,100,100,100,100,100
Paris 12e Arrondissement,78,78,78,78,78,78
Paris 13e Arrondissement,100,100,100,100,100,100
Paris 14e Arrondissement,100,100,100,100,100,100
Paris 15e Arrondissement,100,100,100,100,100,100
Paris 16e Arrondissement,100,100,100,100,100,100
Paris 17e Arrondissement,100,100,100,100,100,100
Paris 18e Arrondissement,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [252]:

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 214 uniques categories.


In [253]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:200]

array(['Garden', 'Historic Site', 'Theater', 'Plaza', 'French Restaurant',
       'Udon Restaurant', 'Art Museum', 'Restaurant',
       'Italian Restaurant', 'Pedestrian Plaza', 'Exhibit', 'Wine Shop',
       'Hotel', 'Cocktail Bar', 'Fountain', 'Bridge', "Women's Store",
       'Israeli Restaurant', 'Pastry Shop', 'Bookstore', 'Cosmetics Shop',
       'Souvlaki Shop', 'Pizza Place', 'Bakery', 'Toy / Game Store',
       'Candy Store', 'Art Gallery', 'Wine Bar', 'Creperie', 'Beer Bar',
       'Boutique', 'Ice Cream Shop', 'Japanese Restaurant',
       'Concert Hall', 'Opera House', 'Indie Movie Theater',
       'Sandwich Place', 'Modern European Restaurant', 'Dessert Shop',
       'Chocolate Shop', 'Clothing Store', 'Seafood Restaurant',
       'Roof Deck', 'Gourmet Shop', 'Souvenir Shop',
       'Szechuan Restaurant', 'Bar', 'Argentinian Restaurant',
       'Corsican Restaurant', 'Coffee Shop',
       'Vegetarian / Vegan Restaurant', 'Bistro', 'Bubble Tea Shop',
       'Vietnamese Rest

In [254]:

# check if the results contain "'Sushi  "
'Sushi Restaurant'  in venues_df['VenueCategory'].unique()

True

###  6. Analyze Each Neighborhood

In [255]:
# one hot encoding
paris_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

print(paris_onehot.shape)
paris_onehot.head()

(1978, 215)


,Neighbourhoods,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cantonese Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,Drive-in Theater,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Forest,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,National Park,Noodle House,Office,Opera House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Radio Station,Record Shop,Recording Studio,Recreation Center,Resort,Restaurant,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Paris 1er Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Paris 1er Arrondissement,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Paris 1er Arrondissement,0,0,0,0,0,0,0,0,0,0,

####  let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [256]:
paris_grouped = paris_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(paris_grouped.shape)
paris_grouped

(20, 215)


,Neighbourhoods,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cantonese Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,Drive-in Theater,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Forest,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,National Park,Noodle House,Office,Opera House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Radio Station,Record Shop,Recording Studio,Recreation Center,Resort,Restaurant,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Paris 10e Arrondissement,0.01,0.00,0.00,0.01,0.01,0.00,0.02,0.00,0.01,0.00,0.050000,0.030000,0.000000,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.000000,0.020000,0.00,0.020000,0.000000,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.010000,0.00,0.000000,0.01,0.02,0.00,0.00,0.000000,0.00,0.010000,0.02,0.01,0.00,0.00,0.00,0.00,0.03,0.07,0.00,0.00,0.00,0.01,0.00,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.050000,0.01,0.000000,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.060000,0.00,0.00,0.02,0.03,0.00,0.000000,0.00,0.00,0.040000,0.020000,0.00,0.00,0.00,0.01,0.01,0.010000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0

####  Create a new DataFrame for 'Sushi Sestaurent ' data only

In [257]:

df_sushi = paris_grouped[["Neighbourhoods","Sushi Restaurant"]]
df_sushi.head()

,Neighbourhoods,Sushi Restaurant
0,Paris 10e Arrondissement,0.00
1,Paris 11e Arrondissement,0.00
2,Paris 12e Arrondissement,0.00
3,Paris 13e Arrondissement,0.00
4,Paris 14e Arrondissement,0.01


## Cluster Neighbourhoods


In [258]:
# set number of clusters
clusters = 10

df_clustering = df_sushi.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int32)

In [259]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.
df_merged = df_sushi.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

df_merged.head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels
0,Paris 10e Arrondissement,0.00,0
1,Paris 11e Arrondissement,0.00,0
2,Paris 12e Arrondissement,0.00,0
3,Paris 13e Arrondissement,0.00,0
4,Paris 14e Arrondissement,0.01,1


In [260]:
# merge df and df_merged to add latitude/longitude for each neighbourhood
df_merged = df_merged.join(df.set_index("Neighbourhood"), on="Neighbourhoods")

df_merged.head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude
0,Paris 10e Arrondissement,0.00,0,75010,48.876155,2.362330
1,Paris 11e Arrondissement,0.00,0,75011,48.859775,2.378126
2,Paris 12e Arrondissement,0.00,0,75012,48.835100,2.444926
3,Paris 13e Arrondissement,0.00,0,75013,48.830090,2.362282
4,Paris 14e Arrondissement,0.01,1,75014,48.829795,2.323826


In [261]:
# sort the results by Cluster Labels
print(df_merged.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged.head()

(20, 6)


,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude
0,Paris 10e Arrondissement,0.0,0,75010,48.876155,2.362330
17,Paris 7e Arrondissement,0.0,0,75007,48.854620,2.313061
16,Paris 6e Arrondissement,0.0,0,75006,48.849175,2.331987
15,Paris 5e Arrondissement,0.0,0,75005,48.845330,2.351901
14,Paris 4e Arrondissement,0.0,0,75004,48.854275,2.361467


####  Finally, let's visualize the resulting clusters

In [262]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i+x+(i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhoods'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 0

In [263]:
df_merged.loc[df_merged['Cluster Labels'] == 0].head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude
0,Paris 10e Arrondissement,0.0,0,75010,48.876155,2.362330
17,Paris 7e Arrondissement,0.0,0,75007,48.854620,2.313061
16,Paris 6e Arrondissement,0.0,0,75006,48.849175,2.331987
15,Paris 5e Arrondissement,0.0,0,75005,48.845330,2.351901
14,Paris 4e Arrondissement,0.0,0,75004,48.854275,2.361467


#### Cluster 1

In [264]:

df_merged.loc[df_merged['Cluster Labels'] == 1].head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude
7,Paris 17e Arrondissement,0.01,1,75017,48.887070,2.306293
4,Paris 14e Arrondissement,0.01,1,75014,48.829795,2.323826


#### Cluster 2

In [265]:

df_merged.loc[df_merged['Cluster Labels'] == 2].head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude


#### Cluster 3

In [266]:

df_merged.loc[df_merged['Cluster Labels'] == 3].head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude


In [267]:

df_merged.loc[df_merged['Cluster Labels'] == 4].head()

,Neighbourhoods,Sushi Restaurant,Cluster Labels,PostalCode,Latitude,Longitude


### Result